# Fetching Microsoft Sentinel Alert Rules

This uses ["Microsoft Sentinel" REST API](https://learn.microsoft.com/en-us/rest/api/securityinsights/), the management API (https://management.azure.com), and the "Azure Service Management" permission (with "user_impersonation").

In [ ]:
appId = ''
tenantId = ''
appSecret = ''

subscriptionId = ''
resourceGroupName = ''
workspaceName = '' 

apiVersion = ''

# Getting the token

In [ ]:
def getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://management.azure.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

In [ ]:
access_token = getaadtoken(tenantId, appId, appSecret)

# Listing alert rules

In [ ]:
def listing_alert_rules(access_token,subscriptionId,resourceGroupName,workspaceName,apiVersion):
    
    url = "https://management.azure.com/subscriptions/"+subscriptionId+\
    "/resourceGroups/"+resourceGroupName+\
    "/providers/Microsoft.OperationalInsights/workspaces/"+workspaceName+\
    "/providers/Microsoft.SecurityInsights/alertRules?api-version="+apiVersion

    req = urllib.request.Request(url, 
                                 headers={'Authorization' : "Bearer " + access_token,\
                                          'Content-Type':'application/json'})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    df_temp = pd.DataFrame.from_records(jsonResponse['value'])
    df_temp = df_temp['properties'].apply(pd.Series)
    return df_temp

In [ ]:
df = listing_alert_rules(access_token,subscriptionId,resourceGroupName,workspaceName,apiVersion)

In [ ]:
df.head()

In [ ]:
df.sort_values(by=['displayName']).to_excel('assignment_2022.xlsx')